In [2]:
#!/usr/bin/env python
import wuml


##	We generated a synthetic data for regression with 4 dimensions where
##	x1 x2 has positive influence
##	x3 has no influence
##	x4 has negative influence
##
##	x1 has normal distribution
##	x2 is exponential distribution but minus 2 so it could be negative
##	x3 is uniform but shouldn't matter
##	x4 is categorical distribution.


data = wuml.wData(xpath='../../data/shap_regress_example_mix_distributions.csv', batch_size=20, 
					label_type='continuous', label_column_name='label', row_id_with_label=0)



#	Example 1
EXP = wuml.explainer(data, 	loss='mse',		# This will create a network for regression and explain instance wise 
						networkStructure=[(100,'relu'),(100,'relu'),(1,'none')], 
						max_epoch=150, learning_rate=0.001, print_network_training_status=False)

# Show the explanation results
explanation = EXP(data)	# outputs the weight importance
print('Notice that since x1 and x2 can be negative, we get both negative and positive influence.')
print('This is not correct since they should only have positive influence.')
print(explanation)



Network Info:
	Learning rate: 0.001
	Max number of epochs: 150
	Cost Function: mse
	Train Loop Callback: None
	Cuda Available: True
	Network Structure
		Linear(in_features=4, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=1, bias=True) , none



  0%|          | 0/30 [00:00<?, ?it/s]

Notice that since x1 and x2 can be negative, we get both negative and positive influence.
This is not correct since they should only have positive influence.
[[  5.7707  -0.4105   0.8504 -37.9866]
 [  2.0252   0.6373   0.3328   0.    ]
 [  4.0259  -1.4754   0.8312 -42.6442]
 [  8.0883   0.9727   1.2552 -12.336 ]
 [  6.4232  -0.3194   0.9843 -41.3612]
 [ -4.3675  -2.0129   0.366  -40.0813]
 [  4.5051   0.7612   1.4451 -13.8043]
 [ -0.615   -0.85     0.0306 -40.2653]
 [ -0.4405  -2.0313   0.3051 -41.1318]
 [  1.8015  -1.9359   0.1402 -41.1875]
 [  0.6655   0.948    0.6191 -13.1796]
 [  5.3013  -1.401    0.1612 -41.4047]
 [  3.3237  -0.0522   0.4924 -28.6058]
 [  0.5113  -1.6883   0.4047 -41.6935]
 [  1.6619  -3.3661   0.0444 -33.444 ]
 [  1.641    0.7564   1.3106 -13.1853]
 [  5.5881  -1.329    0.7302 -41.8859]
 [ -0.9981   0.8197   0.5414 -12.846 ]
 [  1.2982  -0.8012   0.5122 -41.6026]
 [ -3.5984  -2.1028   0.0422 -40.0987]
 [-12.0207  -1.4311   0.0813 -38.1562]
 [  2.789   -1.6861   0

In [3]:
#	Example 2
Cdata = wuml.center_and_scale(data)
EXP2 = wuml.explainer(Cdata, 	loss='mse',		# This will create a network for regression and explain instance wise 
						networkStructure=[(100,'relu'),(100,'relu'),(1,'none')], 
						max_epoch=150, learning_rate=0.001, print_network_training_status=False)

# Show the explanation results
explanation = EXP2(Cdata)	# outputs the weight importance
print('Notice that centering does not help since x1, x2, x4 can be both positive and negative, which is not correct.')
print(explanation)

Network Info:
	Learning rate: 0.001
	Max number of epochs: 150
	Cost Function: mse
	Train Loop Callback: None
	Cuda Available: True
	Network Structure
		Linear(in_features=4, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=1, bias=True) , none



  0%|          | 0/30 [00:00<?, ?it/s]

Notice that centering does not help since x1, x2, x4 can be both positive and negative, which is not correct.
[[  1.1484   3.8267  -3.3385 -12.6421]
 [ -0.15     1.9452  -3.7451  21.0429]
 [  1.1304  -2.1691  -7.0074 -13.716 ]
 [  0.8319  -1.0617  -0.0325  16.8735]
 [  0.9595   2.3005  -7.155  -13.3308]
 [ -7.2198  -6.432    0.2    -14.26  ]
 [  1.233    1.7575  -3.6329  15.0025]
 [ -2.4326   2.2691  -8.57   -14.3686]
 [ -2.1586  -4.5926  -1.8872 -14.891 ]
 [ -0.109   -5.5311  -5.7571 -14.6975]
 [ -1.2801  -2.7374  -0.9635  16.3623]
 [  2.1424  -2.3594  -5.2349 -14.9297]
 [  1.0369   0.6144  -1.0328   6.2908]
 [ -1.0877  -1.2193  -0.0955 -15.5555]
 [  0.0032   0.9637  -5.3322 -13.092 ]
 [ -0.3537  -4.9325  -1.93    15.3835]
 [  1.2721  -4.3411  -0.1319 -14.8997]
 [ -3.1956  -4.8033  -1.3861  16.9366]
 [ -0.3762   1.7035   0.118  -15.1711]
 [ -6.4425  -3.8678  -6.527  -14.04  ]
 [-20.3095  -0.8225   0.0778 -13.3596]
 [  0.5112  -3.7524  -5.66   -13.8274]
 [  1.1124  -5.2371  -1.1131 -15

In [4]:
#	Example 3
Udata = wuml.use_reverse_cdf_to_map_data_between_0_and_1(data, output_type_name='wData')
EXP3 = wuml.explainer(Udata, 	loss='mse',		# This will create a network for regression and explain instance wise 
						networkStructure=[(600,'relu'),(600,'relu'),(600,'relu'),(1,'none')], 
						max_epoch=600, learning_rate=0.001, print_network_training_status=False)

# Show the regression results
Ŷ = EXP3.net(Udata, output_type='ndarray')
SR_train = wuml.summarize_regression_result(Udata.Y, Ŷ)
print(SR_train.true_vs_predict())

# Show the explanation results
explanation = EXP3(Udata)	# outputs the weight importance
print('If we map all the data into the range of [0,1], notice that x1, x2, x4 all have the correct attribution sign')
print('We noticed that it requires a larger network, and longer training time, but it gives better attributions.')
print(explanation)


Network Info:
	Learning rate: 0.001
	Max number of epochs: 600
	Cost Function: mse
	Train Loop Callback: None
	Cuda Available: True
	Network Structure
		Linear(in_features=4, out_features=600, bias=True) , relu
		Linear(in_features=600, out_features=600, bias=True) , relu
		Linear(in_features=600, out_features=600, bias=True) , relu
		Linear(in_features=600, out_features=1, bias=True) , none



UnboundLocalError: local variable 'X' referenced before assignment